In [1]:
import django, sys, os
sys.path.append('/home/galm/software/django/tmv/BasicBrowser')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
from pandas_ods_reader import read_ods
import pandas as pd
from scoping.models import *
import random
import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC

django.db.connection.close()

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('../literature_identification/jupyter-google-e4b72bf44320.json', scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1o18FIQTmGSWJ_m1ePAzrcVhFlfesESCb2bIEG7ibJAA'
wb = gc.open_by_key(spreadsheet_key)
ws = wb.worksheet("category_label_numbers")

In [3]:
table = ws.get_all_values()
category_map = pd.DataFrame(table[1:], columns=table[0])
category_map.head()

,Broad category,Specific category,Labelled documents,Proposed Aggregation 1,Proposed Aggregation 2,Proposed Aggregation 3
0,Physical systems,01 CO2 concentration,5,-,,
1,Physical systems,02 Air or land surface temperature changes,47,-,,
2,Physical systems,03 Extreme temperature,16,-,,
3,Physical systems,04 Radiation,1,-,,
4,Physical systems,05 Changes in precipitation,55,-,,


In [4]:
df = pd.read_csv('../data/1_predicted_category_documents.csv')

print(df.shape)

seen_index = df[df['seen']==1].index
unseen_index = df[df['seen']==0].index
new_index = df[(df['seen']==1) & (df['ar5']==0)].index
rel_index = df[df['relevant']==1].index
r_index = df[df["random_sample"]==1].index
physical_index = df[df['physical_tags']==1].index

new_rel_index = physical_index.intersection(rel_index)

(97122, 274)


In [5]:
from alterations import postfix_data
df = postfix_data(df)

In [6]:
impact_columns = [x for x in df.columns if re.match("^4 - ",x)]
for x in impact_columns:
    row = category_map.loc[category_map['Specific category']==x.split(' - ')[1]]
    if row.shape[0]==0:
        continue
    for i, aggscheme in enumerate(row[['Proposed Aggregation 1', 'Proposed Aggregation 2']].values[0]):
        if aggscheme not in ['-','']:
            nc = f'{17+i} - {aggscheme}'
            if nc not in df.columns:
                df[nc] = 0
            df.loc[df[x]==1,nc] = 1


In [7]:
specific_impacts_1 = [x for x in df.columns if "17 - " in x]
specific_impacts_1

['17 - Other rivers, lakes and soil',
 '17 - Water level',
 '17 - Evapotranspiration',
 '17 - Drought',
 '17 - Floods',
 '17 - Runoff',
 '17 - Snow',
 '17 - Permafrost',
 '17 - Sea ice',
 '17 - Glaciers',
 '17 - Distribution and range shifts (CMES)',
 '17 - Phenology (CMES)',
 '17 - Food/Agriculture',
 '17 - Corals',
 '17 - Physiology, metabolism, mortality and growth (CMES)',
 '17 - Community composition and interaction (CMES)',
 '17 - Ecosystem productivity and carbon cycle (CMES)',
 '17 - Distribution and range shifts (TES)',
 '17 - Phenology (TES)',
 '17 - Physiology, metabolism, mortality and growth (TES)',
 '17 - Community composition and interaction (TES)',
 '17 - Ecosystem productivity and carbon cycle (TES)',
 '17 - Wildfires',
 '17 - Livelihoods and wellbeing',
 '17 - Health',
 '17 - Displacement and migration']

In [8]:
specific_impacts_2 = [x for x in df.columns if "18 - " in x and df[x].sum() > 5]
specific_impacts_2

['18 - Terrestrial Water Cycle',
 '18 - Drought & Floods',
 '18 - Cryosphere',
 '18 - Coastal and marine Ecosystems',
 '18 - Food/Agriculture',
 '18 - Carbon Cycle',
 '18 - Terrestrial ES',
 '18 - Wildfires',
 '18 - Livelihoods and wellbeing',
 '18 - Health',
 '18 - Displacement and migration',
 '18 - Other anthropogenic']

In [9]:
revectorize = True

X_exists = os.path.isfile(f'../data/X_{df.shape[0]}.npz')

if revectorize is True or X_exists is False:
    print("running vectorisation again")
    vec = TfidfVectorizer(
        ngram_range=(1,2),
        min_df=5, max_df=0.8, strip_accents='unicode', 
        max_features=50000,
        #tokenizer=snowball_stemmer()
    )

    vec.fit(df.loc[seen_index,"content"].astype("str"))

    X = vec.transform(df['content'].astype("str"))   
    
    with open (f'../data/vec_{df.shape[0]}.pickle','wb') as f:
        pickle.dump(vec, f)
    import scipy.sparse
    scipy.sparse.save_npz(f'../data/X_{df.shape[0]}.npz', X)
else:
    print("loading feature matrix")
    with open (f'../data/vec_{df.shape[0]}.pickle','rb') as f:
        vec = pickle.load(f)
        X = scipy.sparse.load_npz(f'../data/X_{df.shape[0]}.npz')
        
X.shape

running vectorisation again


(97122, 20749)

In [10]:
y = np.matrix(df[specific_impacts_2])

from sklearn.model_selection import KFold
from sklearn.multiclass import OneVsRestClassifier

kf = KFold(n_splits=10)
kfs = kf.split(X[new_rel_index],y[new_rel_index])
y_preds = []
for k_train, k_test in kfs:
    clf = OneVsRestClassifier(SVC(kernel='linear', class_weight="balanced", probability=True))
    k_train = rel_index[k_train]
    clf.fit(X[k_train],y[k_train])
    y_preds.append(clf.predict(X[unseen_index]))
    
y_preds = np.array(y_preds)
np.save("../data/y_preds_specific_cats.npz",y_preds)

/home/galm/software/django/tmv/venv/lib/python3.6/site-packages/sklearn/multiclass.py:77: UserWarning: Label not 11 is present in all training examples.
  str(classes[c]))


In [11]:
for i, c in enumerate(specific_impacts_2):
    mean_pred = np.mean(y_preds[:,:,i], axis=0)
    std_pred = np.std(y_preds[:,:,i], axis=0)
    preds_upper = np.minimum(mean_pred + std_pred, 1)
    preds_lower = np.maximum(mean_pred - std_pred, 0)
    
    print(c)
    print(np.where(preds_upper>=0.5,1,0).sum())


    df.loc[unseen_index,f'{c} - mean_prediction'] = mean_pred
    df.loc[unseen_index,f'{c} - std_prediction'] = std_pred
    df.loc[unseen_index,f'{c} - lower_pred'] = preds_lower
    df.loc[unseen_index,f'{c} - upper_pred'] = preds_upper

18 - Terrestrial Water Cycle
4430
18 - Drought & Floods
1385
18 - Cryosphere
1075
18 - Coastal and marine Ecosystems
908
18 - Food/Agriculture
565
18 - Carbon Cycle
9
18 - Terrestrial ES
16895
18 - Wildfires
668
18 - Livelihoods and wellbeing
102
18 - Health
2005
18 - Displacement and migration
315
18 - Other anthropogenic
0


In [13]:
basic_cols = [x for x in df.columns if " - " not in x]
cat_cols = [x for x in df.columns if x not in basic_cols and "12 - " in x or "2 - " in x or "6 - " in x or "18 - " in x or "4 - " in x and "hidden" not in x]
cat_cols

['12 - Coastal and marine Ecosystems',
 '12 - Human and managed',
 '12 - Mountains, snow and ice',
 '12 - Rivers, lakes, and soil moisture',
 '12 - Terrestrial ES',
 '14 - Coastal human systems',
 '14 - Cryosphere',
 '14 - Droughts',
 '14 - Marine (eco-)systems',
 '14 - River floods',
 '14 - Terrestrial (eco-)systems',
 '16 - Climate Impact attribution',
 '16 - Climate attribution',
 '16 - Climate event attribution',
 '16 - Experimental evidence',
 '16 - Future/modelled impacts',
 '16 - Impact event attribution',
 '16 - No Climate Impact attribution',
 '16 - Unclear',
 '16 - Weather sensitivity',
 '2 - 2.1. Climate change attribution',
 '2 - 2.2 Trend attribution',
 '2 - 2.3. Attribution to extreme event',
 '2 - 2.3. Impact trend attribution<hidden>',
 '2 - 2.4. Sensitivity',
 '2 - 2.5. Detection of a regional climate trend (no attribution)',
 '2 - 2.6. Null results',
 '4 - 01 CO2 concentration',
 '4 - 02 Air or land surface temperature changes',
 '4 - 03 Extreme temperature',
 '4 - 04

In [14]:
df[basic_cols+cat_cols].to_csv('../data/1_predicted_category_documents_specific.csv')